In [7]:
import numpy as np
import pandas as pd
import time
import cv2
import threading

In [62]:
def auto_canny(gray, sigma=0.33):
    # compute the median of the single channel pixel intensities
    edged = np.zeros((260,680),np.uint8)
    for j in range(0,680,40) :
        for i in range(0,260,40):
            image = gray[i:i+40,j:j+40]
            v = int(np.mean(image))
            # apply automatic Canny edge detection using the computed median
            lower = int(max(0, (1.0 - sigma) * v))
            upper = int(min(255, (1.0 + sigma) * v))
#             print(lower,upper)
            edged[i:i+40,j:j+40] = cv2.Canny(image, 40, v+40)
    return edged

In [63]:
def hod_rd_sub1(meanhls,meanc):
    for i in range(my,shape[1]-lmd,lmd-2):
        ver_rd(i,meanhls,meanc)

In [64]:
def hod_rd_sub2(mean1hls,mean1c):
    for i in range(my+lmd,0, -(lmd-2)):
        ver_rd(i,mean1hls,mean1c)

In [65]:
def hor_rd(meanhls,meanc):
    hod_rd_sub1(meanhls,meanc)
    hod_rd_sub2(meanhls,meanc)

In [66]:
def ver_rd(x,meanhls,meanc):
    lm1 = lim1
    lm2 = lim2
    lmc = limc
    for i in range(mx,lmd+1,-(lmd-10)):
        lm1 += lm1*0.01
        lm2 += lm2*0.01
        lmc += limc*0.02
        lowhls = meanhls - [0,lm1,lm2]
        highhls = meanhls + [180,lm1,lm2]
        lowc = meanc - lmc
        highc = meanc + lmc
        newscr[i-lmd:i, x:x+lmd] =  cv2.inRange(scr[i-lmd:i, x:x+lmd],lowc,highc)#cv2.inRange(hls[i-lmd:i, x:x+lmd],lowhls,highhls)#+
        
        reshls = [0]+[np.mean(hls[i-lmd:i, x:x+lmd,j]) for j in range(1,3)]
        reshls = np.array(reshls)
        
        resc = [np.mean(scr[i-lmd:i, x:x+lmd,j]) for j in range(3)]
        resc = np.mean(resc)
        
        if(abs(reshls[1]-meanhls[1])<=15 and abs(reshls[2]-meanhls[2])<=15):
            meanhls = (4*reshls+6*meanhls)/10
        
        
        if resc <= 0.8*np.mean(highc) and resc >= 0.8*np.mean(lowc):
            meanc = (5*resc+5*meanc)/10
      

In [82]:
import numpy as np
import pandas as pd
import time
import cv2
import threading
# lim = 4*std
lmd = 40
lim1 = lim2 = 10
limc = 25
kernel = np.ones((3,3),np.uint8)

shape = [480,680]#scr.shape
scr2 = 0
mx = shape[0] - 220
my = shape[1]//2
print("running")
kkk = 300*8*2
mask_prev1 = np.zeros((shape[0]-220,shape[1]),np.uint8)
mask_prev2 = np.zeros((shape[0]-220,shape[1]),np.uint8)
mean_prev_hls = 0
mean_prev_c = 0
vrec = cv2.VideoCapture('v4.mp4',0)

last = None

while(kkk):
    ret, scr = vrec.read()
    kkk-=1

while True:
    start_time = time.time()
    blank = cv2.imread('blank.png', 0)
    blank = cv2.resize(blank,(680,260))
    ret, scr = vrec.read()
    if not ret:
            break
    scr = cv2.resize(scr, (680,480)) 
    scr = np.array(scr[220:480,:])
#     scr = cv2.GaussianBlur(scr, (5, 5), 0)
    
    gray = cv2.cvtColor(scr, cv2.COLOR_BGR2GRAY)
    mean1 = np.mean(gray[440-220:480-220,200:480])
    mean1 = np.array(mean1)
    low1 = mean1 - 10
    high1 = mean1 + 1
    canny = cv2.Canny(gray,min(50,mean1-1),high1)
    im2, contours, hierarchy = cv2.findContours(canny, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    scr = cv2.drawContours(scr, contours, -1, 0, 3)
    blank = cv2.drawContours(blank, contours, -1, 0, 3)
    cv2.imshow("edge normal canny", scr)
    
    
    scr1 = scr
    scr2 = scr
    newscr = np.zeros((shape[0]-220,shape[1]),np.uint8)
#     scr = cv2.bilateralFilter(scr,7,12,12)

###   HLS
    hls = cv2.cvtColor(scr, cv2.COLOR_BGR2HLS)
    meanhls = [0]+[np.mean(hls[440-220:480-220,200:480, i]) for i in range(1,3)]
    std = [np.std(hls[440-220:480-220,200:480, i]) for i in range(1,3)]
    lim1 = (lim1 + 16*std[0])/5  #(1:4(4))/5
    lim2 = (lim2 + 16*std[1])/5
#     print(lim1,lim2)
    meanhls = np.array(meanhls)
    meanhls = 0.3*meanhls + mean_prev_hls*0.7
    mean_prev_hls = meanhls


#### RGB
    meanc = [np.mean(scr[440-220:480-220,200:480, i]) for i in range(3)]
    meanc = np.array(meanc)
    meanc = 0.2*meanc + mean_prev_c*0.8
    mean_prev_c = meanc

####
    hor_rd(meanhls,meanc)


    
    mask4 = newscr
    cv2.imshow('newscr',newscr)
    mask3 = newscr #+ mask1
    mask4 = mask_prev1 + mask3 
    mask_prev1 = mask3 
    
    cv2.imshow('band3',mask4)
    
    dist_transform = cv2.distanceTransform(mask4,cv2.DIST_L2,5)
    ret, mask4 = cv2.threshold(dist_transform,0.03*dist_transform.max(),255,0)

    cv2.imshow('band4',mask4)
#     mask4 = np.array(mask4)
    
#     mask4 = np.uint8(mask4)
            
            
            
            
#     for i in range(min(ncont,len(last))):
#         print(len(contours[i]),len(last[i]))
#         if(True):#len(contours[i])>100):
#             epsilon = 0.003*cv2.arcLength(contours[i], True)
#             contours[i] = cv2.approxPolyDP(contours[i], epsilon, True)
#             for k in range(len(contours[i])):
#                 [point.append([i,j] for i incontours[i][k][0][0]:contours[i][k+1][0][0], contours[i][k][0][1]:contours[i][k+1][0][1]]
                
#             print(len(contours[i]))
#     last = contours
        

    
#     cv2.line(scr2,(0,480),(340,240),(255,255,0),2)
#     cv2.line(scr2,(680,480),(340,240),(255,255,0),2)

#     cv2.imshow('band4',mask4)

    cv2.imshow('blank',blank)



#     cv2.line(scr2,(0, 480),(340, int(220)),(0, 255, 255), 1)
#     cv2.line(scr2,(680, 480),(340, int(220)),(0, 255, 255), 1)
    
#     cv2.line(scr2,(0,int(220)),(680,int(220)),(0,0,255),2)#216  220
    
#     for ind in range(0,260+20,20):
#         xx = (ind)*2
#         cv2.putText(scr2,str(xx),(10,480-ind), cv2.FONT_HERSHEY_SIMPLEX, 1,(255,255,0),1,cv2.LINE_AA)
#         cv2.line(scr2,(0,480-ind),(680,480-ind),(0,255,255),1)

    print("FPS: ", 1.0 / (time.time() - start_time))
    q=cv2.waitKey(1)
    if q == 27:
        cv2.destroyAllWindows()
        break
cv2.destroyAllWindows()

running
FPS:  3.043092846783327
FPS:  14.820443238353686
FPS:  16.801342728157636
FPS:  18.543851659939076
FPS:  18.893942124040507
FPS:  18.21923175493999
FPS:  18.075702138846154
FPS:  14.461671074271884
FPS:  13.965325617556278
FPS:  16.16819252400575
FPS:  16.59421499701294
FPS:  17.01805552174372
FPS:  16.64445978872513
FPS:  17.096998255368412
FPS:  17.071529162766087
FPS:  17.072154541865263
FPS:  16.665225683407503
FPS:  15.678468899521532
FPS:  15.531813098509144
FPS:  16.055735470880514
FPS:  16.17342952882973
FPS:  16.48620156988833
FPS:  15.330450705975665
FPS:  15.335327195747077
FPS:  16.21300347893313
FPS:  16.440900930564375
FPS:  14.102340469169757
FPS:  14.559208014273516
FPS:  15.776304158940198
FPS:  16.996401594968717
FPS:  17.011912342679608
FPS:  16.93252108337673
FPS:  16.58397947120156
FPS:  14.040592246432361
FPS:  14.194066268016257
FPS:  13.892748073903798
FPS:  11.170690913352189
FPS:  17.441383898868928
FPS:  17.64009908693659
FPS:  10.72421287323641
FPS: 

In [81]:
import numpy as np
import pandas as pd
import time
import cv2
import threading

vrec = cv2.VideoCapture('v2.mp4',0)
shape = [480,680]

mx = shape[0]
my = shape[1]//2
print("running")
kkk = 300*6*6
while(kkk):
    kkk-=1
    ret, scr = vrec.read()

ret, scr1 = vrec.read()
scr1 = cv2.resize(scr1, (680,480)) 
scr1 = cv2.cvtColor(scr1, cv2.COLOR_BGR2GRAY)
ret, scr2 = vrec.read()
scr2 = cv2.resize(scr2, (680,480)) 
scr2 = cv2.cvtColor(scr2, cv2.COLOR_BGR2GRAY)
while True:
    start_time = time.time()
    scr = scr1
    scr1 = scr2
    ret, scr2 = vrec.read()
#     ret, scr3 = vrec.read()
#     scr = np.mean(scr1,scr2,scr3)
    if not ret:
            break
    scr2 = cv2.resize(scr2, (680,480)) 
    scr2 = cv2.cvtColor(scr2, cv2.COLOR_BGR2GRAY)
    
    d0 = abs(scr1-scr)
    d1 = abs(scr2-scr)
    d2=d0+d1
    
    cv2.imshow('d0',d0)
    cv2.imshow('d1',d1)
    cv2.imshow('d2',d2)
#     cv2.imshow('d3',d3)
#     cv2.imshow('d4',d4)

    cv2.imshow('scr1',scr)
    print("FPS: ", 1.0 / (time.time() - start_time))
    q=cv2.waitKey(1)
    if q == 27:
        cv2.destroyAllWindows()
        break
cv2.destroyAllWindows()

running


error: /Users/travis/build/skvark/opencv-python/opencv/modules/imgproc/src/resize.cpp:4044: error: (-215) ssize.width > 0 && ssize.height > 0 in function resize


In [23]:
img = cv2.imread("c1.png",cv2.IMREAD_GRAYSCALE)
resize_pic=img
#resize_pic=cv2.resize(img,(640,480),interpolation=cv2.INTER_CUBIC)
edges = cv2.Canny(resize_pic,50,150)
lines_data = cv2.HoughLines(edges,1,np.pi/180,150)
# cv2.imshow('img',lines_data)
print(lines_data)
cv2.imshow('img3',resize_pic )

[[[  6.60000000e+01   1.57079637e+00]]

 [[  6.50000000e+01   0.00000000e+00]]

 [[  1.28800000e+03   0.00000000e+00]]

 [[  5.80000000e+01   0.00000000e+00]]

 [[  5.00000000e+00   0.00000000e+00]]

 [[  2.16000000e+02   0.00000000e+00]]

 [[ -9.00000000e+01   2.28638124e+00]]

 [[  7.00000000e+00   0.00000000e+00]]

 [[ -1.02000000e+02   2.30383468e+00]]

 [[  5.60000000e+01   0.00000000e+00]]

 [[  5.20000000e+01   3.49065848e-02]]

 [[  3.26000000e+02   1.83259571e+00]]

 [[  4.00000000e+01   1.74532924e-02]]

 [[  2.00000000e+01   1.74532924e-02]]

 [[ -2.00000000e+00   3.12413931e+00]]

 [[ -1.60000000e+01   3.10668612e+00]]

 [[ -2.90000000e+01   3.10668612e+00]]

 [[  3.10000000e+01   0.00000000e+00]]

 [[  2.60000000e+01   0.00000000e+00]]

 [[  3.30000000e+01   1.74532924e-02]]

 [[ -2.20000000e+01   3.12413931e+00]]

 [[  4.90000000e+01   3.49065848e-02]]

 [[  4.80000000e+01   5.23598790e-02]]

 [[ -2.50000000e+01   3.12413931e+00]]

 [[  3.50000000e+01   0.00000000e+00]]



In [22]:
cv2.destroyAllWindows()